In [2]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import os
from keras.utils import plot_model
import cv2

In [268]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD

from six.moves import cPickle as pickle

In [270]:
import tensorflow as tf

In [3]:
np.random.seed(1337)  

In [276]:
pickle_files = ['open_eyes.pickle', 'closed_eyes.pickle']

In [277]:
i = 0
for pickle_file in pickle_files:
    with open(pickle_file, 'rb') as f:
        save = pickle.load(f)
        if i == 0:
            train_dataset = save['train_dataset']
            train_labels = save['train_labels']
            test_dataset = save['test_dataset']
            test_labels = save['test_labels']
        else:
            print("here")
            train_dataset = np.concatenate((train_dataset, save['train_dataset']))
            train_labels = np.concatenate((train_labels, save['train_labels']))
            test_dataset = np.concatenate((test_dataset, save['test_dataset']))
            test_labels = np.concatenate((test_labels, save['test_labels']))
        del save  
    i += 1

here


In [278]:
print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (3876, 24, 24, 1) (3876, 1)
Test set (970, 24, 24, 1) (970, 1)


In [279]:
X_train = train_dataset

Y_train = train_labels

X_test = test_dataset

Y_test = test_labels

In [280]:
X_train.shape

(3876, 24, 24, 1)

In [281]:
print("{1} train samples, {4} channel{0}, {2}x{3}".format("" if X_train.shape[1] == 1 else "s", *X_train.shape))
print("{1}  test samples, {4} channel{0}, {2}x{3}".format("" if X_test.shape[1] == 1 else "s", *X_test.shape))

3876 train samples, 1 channels, 24x24
970  test samples, 1 channels, 24x24


In [282]:
_, img_rows, img_cols, img_channels, = X_train.shape
print(_, img_channels, img_rows, img_cols)

3876 1 24 24


In [300]:
batch_size = 30
noOfClasses = 1
epochs = 12

In [283]:
model = Sequential()

In [284]:
model.add(Convolution2D(32, (3, 3), padding='same',input_shape=(img_rows, img_cols, img_channels)))
model.add(Activation('relu'))
model.add(Convolution2D(24, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [285]:
model.add(Convolution2D(32, (3, 3), padding='same',input_shape=(img_rows, img_cols, img_channels)))
model.add(Activation('relu'))
model.add(Convolution2D(24, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


In [286]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(noOfClasses))
model.add(Activation('sigmoid'))

In [287]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [288]:
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [291]:
print(X_train.shape, Y_train.shape)

(3876, 24, 24, 1) (3876, 1)


In [292]:
print(X_test.shape, Y_test.shape)

(970, 24, 24, 1) (970, 1)


In [293]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=(X_test, Y_test))

Epoch 1/12
130/130 - 5s - loss: 0.6635 - accuracy: 0.5782 - val_loss: 0.6042 - val_accuracy: 0.6608
Epoch 2/12
130/130 - 6s - loss: 0.6124 - accuracy: 0.6672 - val_loss: 0.5639 - val_accuracy: 0.7258
Epoch 3/12
130/130 - 6s - loss: 0.5640 - accuracy: 0.7072 - val_loss: 0.4711 - val_accuracy: 0.7691
Epoch 4/12
130/130 - 5s - loss: 0.4636 - accuracy: 0.7807 - val_loss: 0.3612 - val_accuracy: 0.8546
Epoch 5/12
130/130 - 6s - loss: 0.3464 - accuracy: 0.8568 - val_loss: 0.2786 - val_accuracy: 0.8825
Epoch 6/12
130/130 - 5s - loss: 0.2730 - accuracy: 0.8916 - val_loss: 0.2221 - val_accuracy: 0.9072
Epoch 7/12
130/130 - 5s - loss: 0.2270 - accuracy: 0.9143 - val_loss: 0.2136 - val_accuracy: 0.9155
Epoch 8/12
130/130 - 5s - loss: 0.2102 - accuracy: 0.9203 - val_loss: 0.1706 - val_accuracy: 0.9423
Epoch 9/12
130/130 - 5s - loss: 0.1879 - accuracy: 0.9291 - val_loss: 0.1602 - val_accuracy: 0.9495
Epoch 10/12
130/130 - 5s - loss: 0.1709 - accuracy: 0.9391 - val_loss: 0.2477 - val_accuracy: 0.8959

In [294]:
score = model.evaluate(X_test, Y_test,  verbose=1)

31/31 [==============================] - 0s 8ms/step - loss: 0.1107 - accuracy: 0.9588


In [295]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.11070766299962997
Test accuracy: 0.9587628841400146


In [297]:
import h5py

In [299]:
model.save("driverDrowsinessModel.h5")